In [ ]:
from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)

In [ ]:
from nanoribbon import plot_b,find_bandgap,get_calc_obj
import matplotlib.pyplot as plt

from aiida.orm.data.array.bands import BandsData
from aiida.orm.querybuilder import QueryBuilder
from optparse import OptionParser
import matplotlib.image as mpimg

In [ ]:
class Foo():
    pass


args = Foo()
args.gap = None
args.vband = None
args.cband = None
args.efermi = None
args.t_magn = None
args.a_magn = None
args.pk = [3726,]
args.formula = None
args.erang = [-3,3]

qb = QueryBuilder()
qb.append(BandsData)
res=[]
names=[]
str_list=[]
hartree_run=[]
bands_run = []
scf_run = []
band_gap_list = []

n_bands=0

for node  in qb.iterall():
    if args.pk is not None:
        if node[0].pk not in args.pk:
            continue
    print node[0]
    bands_calc_obj,scf_calc_obj, hartree_calc_obj,struct=get_calc_obj(node[0])
    s=struct.get_ase()
# thelabel= ''.join(['%s%s' % (key, value) for (key, value) in  struct.get_composition().items()])
    thelabel=s.get_chemical_formula()
    print thelabel
 
    try:
        a_magn=scf_calc_obj.res['absolute_magnetization']
    except:
        print "Absolute magnetization not found!"
        print "Skipping"
        continue
    try:
        t_magn=scf_calc_obj.res['total_magnetization']
    except:
        print "Total magnetization not found!"
        print "Skipping"
        continue
    try:
        fermi_energy= scf_calc_obj.res['fermi_energy']
    except:
        print "Fermi energy not found!"
        print "Skipping"
        continue
 
    vac_lev=0.0
    if hartree_calc_obj != None:
        vac_lev=hartree_calc_obj.res['vacuum_level']*27.211385/2.0
    else:
        print "Hartree calc object was not found!!"
        print "Vacuum level set to 0"
    out=find_bandgap(node[0], fermi_energy=fermi_energy)
    fermi_energy-=vac_lev
    gap=out[1]
    print "band gap: {}".format(gap)

    
    if out[0] == False:
        homo=fermi_energy
        lumo=fermi_energy
    else:
        homo=out[2]-vac_lev
        lumo=out[3]-vac_lev
        
    if args.formula is not None:
        givenformula=args.formula
        if givenformula != thelabel :
            print 'formula'+' '+thelabel+' does not match'+' '+givenformula
            continue
    if args.gap is not None:
        if gap < args.gap[0] or gap > args.gap[1]:
            continue
    if args.vband is not None:
        if homo < args.vband[0] or homo > args.vband[1]:
            continue
    if args.cband is not None:
        if lumo < args.cband[0] or lumo > args.band[1]:
            continue
    if args.efermi is not None:
        if fermi_energy < args.efermi[0] or fermi_energy > args.efermi[1]:
            continue
    if args.t_magn is not None:
        if t_magn < args.t_magn[0] or t_magn > args.t_magn[1]:
            continue
    if args.a_magn is not None:
        if a_magn < args.a_magn[0] or a_magn > args.a_magn[1]:
            continue
    res.append(node[0])
    s=struct.get_ase()
    formulatoappend=s.get_chemical_formula()
    names.append(formulatoappend)
    str_list.append(struct)
    hartree_run.append(hartree_calc_obj)
    bands_run.append(bands_calc_obj)
    scf_run.append(scf_calc_obj)
    band_gap_list.append(gap)
    allign=(homo+lumo)/2.0
    n_bands+=1


print "Bands found: {}".format(n_bands)




